# <font color="#1d479b">**Projet 8: I'm something of a painter my self** </font>

La vision par ordinateur a énormément progressé ces dernières années et les GANs sont désormais capables de mimer des objets de manière très convaincante.

Dans ce challenge kaggle, le but est de générer des peinture style Monet à partir des photogaphies réelles prises par une caméra.

Dataset

    Monet_jpg : 300 peintures de Monet format 256x256 au format jpeg
    monet_tfrec : 300 peintures de Monet format 256x256 au format tfrecord
    photo_jpg : 7028 photos de taille 256x256 au format jpeg
    photo_tfrec : 7028 photos de taille 256x256 au format tfrecord



## <font color="#1d479b" id="section_1">**1. Importation des librairies**</font>

In [ ]:
from __future__ import print_function
%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import itertools
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.animation as animation
from IPython.display import HTML

In [ ]:
# Set random seed for reproducibility
manualSeed = 999
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
class Arguments():
    # main parameters

    # Root directory for dataset
    dataroot_A = "train_photo/photo_jpg"
    dataroot_B = "train_monet/monet_jpg"
    
    dataroot_test_A = "test_photo"

    # Number of workers for dataloader
    workers = 4

    # Batch size during training
    batch_size = 4

    # Spatial size of training images. All images will be resized to this
    #   size using a transformer.
    image_size = 256

    # Number of channels in the group A
    input_nc = 3
    
    # Number of channels in the group B
    output_nc = 3

    # number of filters in the last conv layer of generator
    ngf = 32

    # number of filters in the first conv layer of discriminator
    ndf = 16

    # Number of training epochs
    n_epochs = 20

    # Learning rate for optimizers
    lr = 0.0002
    
    # identity_loss coeff.
    lambda_identity = .5
    lambda_A = 10.0
    lambda_B = 10.0

    # Beta1 hyperparam for Adam optimizers
    beta1 = 0.5

    # Number of GPUs available. Use 0 for CPU mode.
    ngpu = 1
    
    gan_mode = 'lsgan' # 'vanilla', 'wgangp'
    
    pool_size = 16

    device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
    print(device)
    
    print_freq = 1
    save_freq = 5
    save_dir = "/content/drive/MyDrive/data/"
opt = Arguments()

In [ ]:
# to empty up the memory in GPU
torch.cuda.empty_cache()

## <font color="#1d479b" id="section_2">**2. Chargement des données**</font>

In [ ]:
# create a dataset to return pairs of images from both 2 styles
class UnpairedDataset(torch.utils.data.Dataset):
    def __init__(self, opt):
        self.path_A = opt.dataroot_A
        self.path_B = opt.dataroot_B
        
        transform = transforms.Compose([
            transforms.Resize(opt.image_size),
            transforms.CenterCrop(opt.image_size),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ])
        
        self.datasetA = dset.ImageFolder(root = self.path_A, transform = transform)
        self.datasetB = dset.ImageFolder(root = self.path_B, transform = transform)
        
        self.size_A = len(self.datasetA)
        self.size_B = len(self.datasetB)

    def __len__(self):
        return max(self.size_A, self.size_B)

    def __getitem__(self, index):
        A_tensor = self.datasetA[index % self.size_A][0]
        
        index_B = random.randint(0, self.size_B - 1)
        B_tensor = self.datasetB[index_B][0]
        
        return {'A': A_tensor, 'B': B_tensor}

In [ ]:
unpaired_dataset = UnpairedDataset(opt)
dataloader = torch.utils.data.DataLoader(unpaired_dataset, batch_size=opt.batch_size, shuffle=True, num_workers=opt.workers)

In [ ]:
# visualize a batch of data
real_batch = next(iter(dataloader))

plt.figure(figsize=(8,4), dpi=128)
plt.subplot(1,2,1)
plt.axis("off")
plt.title("(A): actual photos")

grid_A = vutils.make_grid(real_batch['A'], padding=2, normalize=True, nrow=4).cpu()
plt.imshow(np.transpose(grid_A,(1,2,0)))

plt.subplot(1,2,2)
# plt.figure(figsize=(16,4), dpi=128)
plt.axis("off")
plt.title("(B): Monet paintings")
grid_B = vutils.make_grid(real_batch['B'], padding=2, normalize=True, nrow=4).cpu()
plt.imshow(np.transpose(grid_B,(1,2,0)))

In [ ]:
dataset_test = dset.ImageFolder(
    root = opt.dataroot_test_A,
    transform = transforms.Compose([
        transforms.Resize(opt.image_size),
        transforms.CenterCrop(opt.image_size),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]))

dataloader_test = torch.utils.data.DataLoader(dataset_test, batch_size=4, shuffle=False, num_workers=0)